<a href="https://colab.research.google.com/github/AdarshShah/final-year-project/blob/master/project_sepsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [0]:
import pandas as pd
import psycopg2
import getpass
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Declare Global Variables

In [0]:
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'
password = 'postgres'
icd9_code = '99592'
minMaxScaler = dict()

# Create Database Connection

In [0]:
conn = psycopg2.connect(user=user,password=password,dbname=dbname,host=host)
cur = conn.cursor()
cur.execute('SET search_path to {}'.format(schema))

# Load Patient data




> Data from admissions suffering from severe sepsis

In [0]:
query = """SELECT DISTINCT * FROM mimiciii.admissions i 
INNER JOIN mimiciii.diagnoses_icd USING(hadm_id)
INNER JOIN mimiciii.d_icd_diagnoses USING(icd9_code)
WHERE icd9_code LIKE '99592';
"""
admissions = pd.read_sql_query(sql=query,con=conn)

> List Items from d_labitems and d_items

In [0]:
#d_items = ['Platelets','C Reactive Protein (CRP)','pH (Art)','HCO3','TCO2 (cap)']
#d_labitems = ['Leukocytes','Urea Nitrogen','Creatinine','Glucose','Calcium, Total','Sodium','Potassium','Bilirubin','Albumin','Lactate','pCO2','pO2','Hemoglobin']

#Append itemid below !!!!
d_item_ce = ['0','828','3789','6256','791','3750','1525','220615','811','3744','3745','1529','226537','837','3803','1536','4948','3066','227444']  
d_item_cv = ['0','30006','44711','44441']
d_item_mv = ['0','225170']
d_labitem = ['51486','51006','50912','50809','50931','51478','50813','50820']

In [0]:
def get_pandas(items, table):
  query = "SELECT DISTINCT * FROM mimiciii."+table+" WHERE itemid IN " + str(tuple(items))
  items = pd.read_sql_query(sql=query,con=conn)
  return items

d_item_ce = get_pandas(d_item_ce,"d_items")
d_item_cv = get_pandas(d_item_cv,"d_items")
d_item_mv = get_pandas(d_item_mv,"d_items")
d_labitem = get_pandas(d_labitem,"d_labitems")

# Fetch items from mimic-iii

> CHARTEVENTS

In [0]:
query = "SELECT * FROM mimiciii.chartevents WHERE itemid IN " + str(tuple(d_item_ce['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
chartevents = pd.read_sql_query(sql=query,con=conn)


In [0]:
itemids = chartevents['itemid'].drop_duplicates()
temp = chartevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=0,inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [8]:
chartevents.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,cgid,valuenum,warning,error
count,3.000570e+05,300057.000000,300057.000000,299723.000000,300057.000000,300057.000000,299216.000000,44258.000000,44258.000000
mean,1.014512e+08,22532.331060,149795.256431,250952.877267,33711.791216,16253.674169,98.363125,0.595305,0.002689
std,5.663463e+07,20133.000158,28668.160671,28597.832480,78331.500177,2026.583433,90.804017,0.490839,0.051784
min,1.565500e+04,21.000000,100074.000000,200030.000000,791.000000,14023.000000,-70.000000,0.000000,0.000000
25%,5.315241e+07,9233.000000,125169.000000,226512.000000,811.000000,15331.000000,2.600000,0.000000,0.000000
50%,1.003898e+08,18748.000000,149299.000000,251770.000000,1525.000000,15331.000000,113.000000,1.000000,0.000000
75%,1.523907e+08,28150.000000,175529.000000,275498.000000,1536.000000,15331.000000,141.000000,1.000000,0.000000
max,1.931220e+08,99991.000000,199943.000000,299998.000000,227444.000000,21570.000000,3555.000000,1.000000,1.000000


> INPUTEVENTS_CV



In [0]:
query = "SELECT * FROM mimiciii.inputevents_cv WHERE itemid IN " + str(tuple(d_item_cv['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
inputevents_cv = pd.read_sql_query(sql=query,con=conn)

In [0]:
itemids = inputevents_cv['itemid'].drop_duplicates()
temp = inputevents_cv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=0,inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [10]:
inputevents_cv.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,amount,cgid,orderid,linkorderid,newbottle
count,2.021000e+03,2021.000000,2021.000000,2011.000000,2021.000000,1923.000000,2021.000000,2.021000e+03,2.021000e+03,237.0
mean,4.758768e+06,16271.764473,149125.329540,252203.299354,30338.563088,210.064621,17932.677387,5.990417e+06,6.012288e+06,1.0
std,1.933361e+06,8929.825919,30072.238069,29738.749552,2179.733312,117.960518,2294.238327,3.441337e+06,3.476388e+06,0.0
min,4.457295e+06,117.000000,100253.000000,200102.000000,30006.000000,0.000000,14151.000000,4.580000e+03,9.591000e+03,1.0
25%,4.460469e+06,10444.000000,121169.000000,226684.000000,30006.000000,198.000000,15816.000000,3.014334e+06,2.921896e+06,1.0
50%,4.463877e+06,15812.000000,152446.000000,257318.000000,30006.000000,232.000000,17609.000000,5.985687e+06,6.254552e+06,1.0
75%,4.466987e+06,22907.000000,180799.000000,276363.000000,30006.000000,270.000000,20034.000000,8.936081e+06,9.199768e+06,1.0
max,1.745546e+07,32658.000000,199716.000000,299994.000000,44711.000000,663.000000,21570.000000,1.195363e+07,1.190499e+07,1.0


> INPUTEVENTS_MV

In [0]:
query = "SELECT * FROM mimiciii.inputevents_mv WHERE itemid IN ('225170') AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,starttime,itemid;"
inputevents_mv = pd.read_sql_query(sql=query,con=conn)

In [0]:
itemids = inputevents_mv['itemid'].drop_duplicates()
temp = inputevents_mv.pivot(index='row_id',columns='itemid',values='amount')
temp.fillna(value=0,inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [12]:
inputevents_mv.describe()

,row_id,subject_id,hadm_id,icustay_id,itemid,amount,rate,cgid,orderid,linkorderid,patientweight,totalamount,isopenbag,continueinnextdept,cancelreason,originalamount,originalrate
count,1.334000e+03,1334.000000,1334.000000,1333.000000,1334.0,1334.000000,1334.000000,1334.000000,1.334000e+03,1.334000e+03,1334.000000,1334.000000,1334.000000,1334.0,1334.000000,1334.000000,1334.000000
mean,1.723259e+06,61245.308846,149024.087706,249418.126032,225170.0,246.171973,325.239816,17902.677661,4.940668e+06,4.941656e+06,86.161694,251.479010,0.000750,0.0,0.116942,250.899538,325.085764
std,1.103053e+06,22845.351072,27850.435266,31289.644167,0.0,81.570682,196.546427,2260.232552,2.878895e+06,2.869574e+06,22.073056,74.877018,0.027379,0.0,0.407872,75.299161,196.324979
min,2.146900e+04,124.000000,100442.000000,200454.000000,225170.0,-20.099999,2.000000,14023.000000,1.289000e+04,1.289000e+04,1.000000,130.000000,0.000000,0.0,0.000000,70.000000,2.000000
25%,7.049760e+05,47183.000000,123675.000000,217745.000000,225170.0,198.999996,200.999994,15830.000000,2.539858e+06,2.544540e+06,71.000000,199.000000,0.000000,0.0,0.000000,199.000000,201.000000
50%,1.646684e+06,60762.000000,143320.000000,246671.000000,225170.0,230.999994,256.000014,17677.000000,4.875910e+06,4.883992e+06,85.700000,233.000000,0.000000,0.0,0.000000,233.000000,256.000000
75%,2.800743e+06,77893.000000,175201.000000,279535.000000,225170.0,281.000004,386.000004,20013.250000,7.440448e+06,7.434026e+06,97.000000,282.000000,0.000000,0.0,0.000000,282.000000,386.000000
max,3.605911e+06,99865.000000,199742.000000,299533.000000,225170.0,757.000020,1899.999960,21492.000000,9.992465e+06,9.992465e+06,170.500000,757.000000,1.000000,0.0,2.000000,757.000000,1900.000000


> LABEVENTS

In [0]:
query = "SELECT * FROM mimiciii.labevents WHERE itemid IN " + str(tuple(d_labitem['itemid'])) + " AND hadm_id IN " + str(tuple(admissions['hadm_id'])) + " ORDER BY hadm_id,charttime,itemid;"
labevents = pd.read_sql_query(sql=query,con=conn)

In [0]:
itemids = labevents['itemid'].drop_duplicates()
temp = labevents.pivot(index='row_id',columns='itemid',values='valuenum')
temp.fillna(value=0,inplace=True)
for item in itemids:
  arr = np.reshape(np.array(temp[item]),(-1,1))
  minMaxScaler[item] = MinMaxScaler().fit(arr)

In [14]:
labevents.describe()

,row_id,subject_id,hadm_id,itemid,valuenum
count,4.336430e+05,433643.000000,433643.000000,433643.000000,417256.000000
mean,1.581387e+07,38336.650607,149173.562802,50924.733484,45.416218
std,8.309446e+06,29619.847083,28457.551458,132.967727,66.878142
min,1.021200e+04,21.000000,100074.000000,50809.000000,0.000000
25%,8.455738e+06,13603.000000,125067.000000,50820.000000,3.000000
50%,1.737978e+07,28067.000000,148392.000000,50912.000000,7.470000
75%,2.304729e+07,62919.000000,174727.000000,50931.000000,84.000000
max,2.790723e+07,99991.000000,199943.000000,51486.000000,3565.000000


# Generate Tensors

> CHARTEVENTS

In [0]:
hadm_id = chartevents['hadm_id'].drop_duplicates()

In [0]:
dataset_ce = list()
for hadm in hadm_id:
  temp = chartevents[chartevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  dataset_ce.append(temp)

> INPUTEVENTS_CV

In [0]:
hadm_id = inputevents_cv['hadm_id'].drop_duplicates()

In [0]:
dataset_cv = list()
for hadm in hadm_id:
  temp = inputevents_cv[inputevents_cv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  dataset_cv.append(temp)

> INPUTEVENTS_MV

In [0]:
hadm_id = inputevents_mv['hadm_id'].drop_duplicates()

In [0]:
dataset_mv = list()
for hadm in hadm_id:
  temp = inputevents_mv[inputevents_mv['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='amount')
  dataset_mv.append(temp)

> LABEVENTS

In [0]:
hadm_id = labevents['hadm_id'].drop_duplicates()

In [0]:
dataset_lab = list()
for hadm in hadm_id:
  temp = labevents[labevents['hadm_id'] == hadm]
  temp = temp.pivot(index='row_id',columns='itemid',values='valuenum')
  dataset_lab.append(temp)